In [1]:
import gym
import neat
import numpy as np
import time
from abc import ABC, abstractmethod

In [2]:
render_slowdown = 0.03

def run(config_file, eval_genomes, visual_reporter):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    p.add_reporter(neat.StatisticsReporter())
    p.add_reporter(neat.Checkpointer(5))
    p.add_reporter(visual_reporter)

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 500)
    return winner

def simulate(actor, turns, render):
    render_mode = "human" if render else None
    env = gym.make(simulation_type, render_mode=render_mode)
    observation = env.reset()[0]
    
    for i in range(turns):
        action = actor.act(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        actor.learn(observation, reward)

        if render == True:
            time.sleep(render_slowdown)
            env.render()
            
        if terminated:
            break
    
    env.close()
    
def createGenomeEvaluator(Actor):
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            genome.fitness = -1000
            net = neat.nn.FeedForwardNetwork.create(genome, config)
            actor = Actor(genome, net, True)
            simulate(actor, num_turns, False)
            
    return eval_genomes

class Actor(ABC):
    def __init__(self, genome, net, isLearning):
        self.genome = genome
        self.net = net
        self.isLearning = isLearning
            
    def learn(self, observation, fitness):
        if self.isLearning == True:
            self.genome.fitness = self.evaluate(observation, fitness)
            
    @abstractmethod
    def act(self, observation):
        pass
    
    @staticmethod
    @abstractmethod
    def evaluate(observation, fitness):
        return fitness
    
class VisualReporter(neat.reporting.BaseReporter):
    def __init__(self, Actor, min_generation, generation_gap):
        neat.reporting.BaseReporter.__init__(self)
        self.Actor = Actor
        self.current_generation = 0
        self.min_generation = min_generation
        self.generation_gap = generation_gap
        
    def end_generation(self, config, population, species_set):
        self.current_generation += 1
        
    def post_evaluate(self, config, population, species, best_genome):
        if self.current_generation > self.min_generation and self.current_generation % self.generation_gap == 0:
            best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)
            best_acrobot = self.Actor(best_genome, best_net, False)
            simulate(best_acrobot, num_turns, True)

In [ ]:
num_turns = 200
render_generation = 1
min_render_generation = 0
simulation_type = "Acrobot-v1"

class Acrobot(Actor):
    def act(self, observation):
        output = self.net.activate(observation)[0]

        if output > 0.75:
            return 1
        elif output <0.25:
            return -1
        else:
            return 0
    
    @staticmethod
    def evaluate(observation, fitness):
        return fitness
        
eval_genomes = createGenomeEvaluator(Acrobot)
visual_reporter = VisualReporter(Acrobot, min_render_generation, render_generation)
best_acrobot_run = run('neat_acrobot.cfg', eval_genomes, visual_reporter)

In [4]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'neat_acrobot.cfg')

best_acrobot_net = neat.nn.FeedForwardNetwork.create(best_acrobot_run, config)
best_acrobot = Acrobot(best_acrobot_run, best_acrobot_net, False)
simulate(best_acrobot, num_turns, True)

In [3]:
num_turns = 500
render_generation = 10
min_render_generation = 20
simulation_type = "MountainCarContinuous-v0"

class MountainCar(Actor):        
    def act(self, observation):
        output = self.net.activate(observation)
        return ([output[0] * 2 - 1])
    
    @staticmethod
    def evaluate(observation, fitness):
        return fitness + 100 * abs(observation[0])
        

eval_genomes = createGenomeEvaluator(MountainCar)
visual_reporter = VisualReporter(MountainCar, min_render_generation, render_generation)
best_car_run = run('neat_mountain.cfg', eval_genomes, visual_reporter)


 ****** Running generation 0 ****** 

Population's average fitness: 52.67941 stdev: 26.09603
Best fitness: 145.42512 - size: (1, 2) - species 1 - id 66
Average adjusted fitness: 0.363
Mean genetic distance 1.199, standard deviation 0.441
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   200    145.4    0.363     0
Total extinctions: 0
Generation time: 2.092 sec

 ****** Running generation 1 ****** 

Population's average fitness: 63.56709 stdev: 23.85264
Best fitness: 145.41587 - size: (1, 1) - species 1 - id 306
Average adjusted fitness: 0.430
Mean genetic distance 1.407, standard deviation 0.478
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   200    145.4    0.430     1
Total extinctions: 0
Generation time: 2.003 sec (2.048 average)

 ****** Running generation 2 ****** 

Population's average fitness: 65.83792

Population's average fitness: 77.78762 stdev: 36.28366
Best fitness: 149.90203 - size: (2, 3) - species 2 - id 3083
Average adjusted fitness: 0.453
Mean genetic distance 1.989, standard deviation 0.795
Population of 200 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   15    24     94.7    0.373     1
     2   11   107    149.9    0.549     0
     3   11    35    107.4    0.446     7
     4    8    34    106.9    0.446     3
Total extinctions: 0
Generation time: 1.660 sec (1.710 average)

 ****** Running generation 16 ****** 

Population's average fitness: 73.58095 stdev: 34.05853
Best fitness: 149.09742 - size: (2, 3) - species 2 - id 3304
Average adjusted fitness: 0.477
Mean genetic distance 1.944, standard deviation 0.775
Population of 199 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   16    29    145.8    0.458     2
     2   12    95    149.1    0.51

In [4]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'neat_mountain.cfg')

best_car_net = neat.nn.FeedForwardNetwork.create(best_car_run, config)
best_mountain_car = MountainCar(best_car_run, best_car_net, False)
simulate(best_mountain_car, num_turns, True)

In [ ]:
num_turns = 250
render_generation = 3
min_render_generation = 4
simulation_type = "LunarLanderContinuous-v2"

class LunarLander(Actor):
    def act(self, observation):
        output = self.net.activate(observation)
        return ([i*2-1 for i in output])
    
    @staticmethod
    def evaluate(observation, fitness):
        return fitness - 10 * observation[1] - abs(observation[0]) - observation[2]

eval_genomes = createGenomeEvaluator(LunarLander)       
visual_reporter = VisualReporter(LunarLander, min_render_generation, render_generation)
best_lunar_run = run('neat_lander.cfg', eval_genomes, visual_reporter)


 ****** Running generation 0 ****** 

Population's average fitness: -104.54914 stdev: 20.11476
Best fitness: -0.27292 - size: (2, 16) - species 1 - id 39
Average adjusted fitness: 0.476
Mean genetic distance 0.974, standard deviation 0.215
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100     -0.3    0.476     0
Total extinctions: 0
Generation time: 2.104 sec

 ****** Running generation 1 ****** 

Population's average fitness: -102.34351 stdev: 13.72769
Best fitness: -3.05844 - size: (2, 16) - species 1 - id 39
Average adjusted fitness: 0.385
Mean genetic distance 1.175, standard deviation 0.251
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100     -3.1    0.385     1
Total extinctions: 0
Generation time: 2.078 sec (2.091 average)

 ****** Running generation 2 ****** 

Population's average fitness: -104.

Population's average fitness: -100.26100 stdev: 2.45161
Best fitness: -96.04099 - size: (2, 10) - species 2 - id 1794
Average adjusted fitness: 0.694
Mean genetic distance 1.928, standard deviation 0.444
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   18    49    -96.4    0.656    18
     2    1    51    -96.0    0.731     0
Total extinctions: 0
Generation time: 4.919 sec (4.029 average)

 ****** Running generation 19 ****** 

Population's average fitness: -101.63697 stdev: 4.20261
Best fitness: -96.46188 - size: (2, 9) - species 2 - id 1930
Average adjusted fitness: 0.818
Mean genetic distance 1.803, standard deviation 0.464
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   19    34    -97.7    0.782    19
     2    2    66    -96.5    0.855     1
Total extinctions: 0
Generation time: 1.465 sec (3.495 average)
Saving ch

Population's average fitness: -101.82713 stdev: 4.91036
Best fitness: -95.24684 - size: (2, 12) - species 1 - id 3324
Average adjusted fitness: 0.720
Mean genetic distance 1.773, standard deviation 0.514
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   34    61    -95.2    0.686    34
     2   17    39    -97.7    0.754    11
Total extinctions: 0
Generation time: 1.431 sec (3.503 average)
Saving checkpoint to neat-checkpoint-34

 ****** Running generation 35 ****** 

Population's average fitness: -101.66523 stdev: 4.56441
Best fitness: -96.80996 - size: (2, 9) - species 1 - id 3431
Average adjusted fitness: 0.838
Mean genetic distance 1.748, standard deviation 0.507
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   35    51    -96.8    0.869    35
     2   18    49    -97.9    0.806    12
Total extinctions: 0
Generation t

Population's average fitness: -100.88319 stdev: 4.24834
Best fitness: -96.72714 - size: (2, 5) - species 1 - id 4860
Average adjusted fitness: 0.848
Mean genetic distance 2.237, standard deviation 0.795
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   50    54    -96.7    0.803    50
     3    4    46    -97.1    0.893     2
Total extinctions: 0
Generation time: 1.362 sec (2.809 average)

 ****** Running generation 51 ****** 

Population's average fitness: -100.80997 stdev: 3.78079
Best fitness: -95.35459 - size: (2, 7) - species 3 - id 4977
Average adjusted fitness: 0.826
Mean genetic distance 2.352, standard deviation 0.707
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   51    44    -96.4    0.805    51
     3    5    56    -95.4    0.847     3
Total extinctions: 0
Generation time: 6.611 sec (3.336 average)

 ****** R

Population's average fitness: -100.91096 stdev: 3.20325
Best fitness: -96.54546 - size: (2, 6) - species 5 - id 6327
Average adjusted fitness: 0.805
Mean genetic distance 2.518, standard deviation 0.916
Population of 100 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   19    27    -98.2    0.827    12
     4   11    27    -97.6    0.797     2
     5    9    36    -96.5    0.790     5
     6    0     4       --       --     0
     7    0     6       --       --     0
Total extinctions: 0
Generation time: 1.316 sec (2.524 average)

 ****** Running generation 66 ****** 

Population's average fitness: -101.28415 stdev: 3.78939
Best fitness: -96.72349 - size: (2, 2) - species 3 - id 6366
Average adjusted fitness: 0.778
Mean genetic distance 2.552, standard deviation 0.829
Population of 100 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   20    23    -96.7    0.

...

Population's average fitness: -101.42685 stdev: 3.80690
Best fitness: -97.00616 - size: (5, 5) - species 17 - id 26490
Average adjusted fitness: 0.754
Mean genetic distance 2.154, standard deviation 0.696
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  120    48    -97.0    0.685     8
    19    5    52    -97.4    0.823     2
Total extinctions: 0
Generation time: 1.422 sec (2.799 average)

 ****** Running generation 282 ****** 

Population's average fitness: -102.65830 stdev: 5.05873
Best fitness: -97.02162 - size: (5, 6) - species 17 - id 26659
Average adjusted fitness: 0.736
Mean genetic distance 2.279, standard deviation 0.629
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  121    52    -97.0    0.672     9
    19    6    48    -97.5    0.800     3
Total extinctions: 0
Generation time: 7.162 sec (3.386 average)

 ***

Population's average fitness: -100.13270 stdev: 1.22923
Best fitness: -96.51705 - size: (7, 6) - species 17 - id 28073
Average adjusted fitness: 0.573
Mean genetic distance 2.213, standard deviation 0.516
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  136    47    -96.5    0.566     1
    19   21    53    -97.5    0.580     8
Total extinctions: 0
Generation time: 5.191 sec (2.790 average)

 ****** Running generation 298 ****** 

Population's average fitness: -100.45207 stdev: 2.37272
Best fitness: -96.51898 - size: (6, 2) - species 19 - id 28103
Average adjusted fitness: 0.801
Mean genetic distance 2.333, standard deviation 0.509
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  137    88    -97.8    0.805     2
    19   22    12    -96.5    0.798     9
Total extinctions: 0
Generation time: 1.171 sec (2.447 average)

 ***

Population's average fitness: -101.13979 stdev: 3.67317
Best fitness: -96.50210 - size: (6, 2) - species 17 - id 29451
Average adjusted fitness: 0.770
Mean genetic distance 2.230, standard deviation 0.595
Population of 99 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  151    20    -96.5    0.739    11
    20   11    44    -96.6    0.747     3
    21    6    35    -97.1    0.825     1
Total extinctions: 0
Generation time: 5.165 sec (3.061 average)

 ****** Running generation 313 ****** 

Population's average fitness: -100.82174 stdev: 3.07872
Best fitness: -96.91181 - size: (7, 3) - species 17 - id 29599
Average adjusted fitness: 0.783
Mean genetic distance 2.170, standard deviation 0.527
Population of 99 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  152    28    -96.9    0.771    12
    20   12    43    -97.0    0.797     4
    21    7    28    -98.3   

Population's average fitness: -101.02010 stdev: 3.60402
Best fitness: -95.22873 - size: (7, 5) - species 17 - id 30861
Average adjusted fitness: 0.744
Mean genetic distance 1.979, standard deviation 0.538
Population of 100 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  166    35    -95.2    0.755    13
    20   26    34    -97.2    0.743    18
    21   21    31    -95.8    0.733     7
Total extinctions: 0
Generation time: 5.771 sec (3.387 average)

 ****** Running generation 328 ****** 

Population's average fitness: -101.54578 stdev: 4.21295
Best fitness: -96.62635 - size: (8, 10) - species 21 - id 30942
Average adjusted fitness: 0.747
Mean genetic distance 2.053, standard deviation 0.519
Population of 100 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  167    34    -97.5    0.722    14
    20   27    30    -97.1    0.722    19
    21   22    36    -96.6

Average adjusted fitness: 0.783
Mean genetic distance 2.209, standard deviation 0.602
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  181    50    -96.7    0.781    28
    21   36    50    -98.4    0.785    22
Total extinctions: 0
Generation time: 5.803 sec (3.049 average)

 ****** Running generation 343 ****** 

Population's average fitness: -102.10730 stdev: 6.52141
Best fitness: -95.91115 - size: (7, 4) - species 17 - id 32464
Average adjusted fitness: 0.883
Mean genetic distance 2.284, standard deviation 0.565
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    17  182    50    -95.9    0.883    29
    21   37    50    -97.5    0.884    23
Total extinctions: 0
Generation time: 1.407 sec (2.743 average)

 ****** Running generation 344 ****** 

Population's average fitness: -101.73785 stdev: 4.88778
Best fitness: -96.27396 - s

In [10]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'neat_lander.cfg')

best_lunar_net = neat.nn.FeedForwardNetwork.create(best_lunar_run, config)
best_lunar_lander = LunarLander(best_lunar_run, best_lunar_net, False)
simulate(best_lunar_lander, num_turns, True)